# Teste 2 Market Sizing

### Utilizando LLM para ler um demonstrativo financeiro de uma empresa e identificar o PDD 

In [1]:
import os
import glob
from dotenv import load_dotenv
from IPython.display import Markdown, display



In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("demonstrativos/BTG.pdf")
pages = loader.load()

# cada página é um "Document"
docs = [p.page_content for p in pages]


In [4]:
print(f"Total de páginas extraídas: {len(pages)}")

pagina_18 = pages[17]  # índice 17 corresponde à página 18
print("\n--- Página 9 ---")
print(pagina_18.page_content[:2000])  # mostra até 2000 caracteres




Total de páginas extraídas: 62

--- Página 9 ---
17 
Demonstrações Financeiras Completas 
Banco BTG Pactual S.A. 
Demonstração dos fluxos de caixa 
Semestre e exercícios findos em 31 de dezembro 
(Em milhares de reais)  
Banco Consolidado 
Nota 31/12/2024 31/12/2023 31/12/2024 31/12/2023 
2º semestre Exercício Exercício Exercício Exercício 
Atividades operacionais 
Lucro líquido do semestre /  exercício  6.192.330 11.789.387 9.924.566 11.789.387 9.924.566 
Ajustes ao lucro líquido (3.634.879) (6.433.844) (2.117.029) 7.009.712 4.053.731 
Resultado de participações em controladas, coligadas e controladas em conjunto  13 (4.751.673) (8.387.822) (4.985.150) (1.390.483) (1.041.735) 
Despesas de juros com dívidas subordinadas e instrumentos de dívida elegíveis a capital  1.279.221 2.266.667 1.519.826 2.334.071 1.579.920 
Provisão / (reversão) para perdas esperadas associadas ao risco de crédito  10 (987.822) (972.142) 159.648 2.844.319 2.410.322 
Provisão para garantias financeiras prestadas

In [5]:
# # Filtra apenas os elementos do tipo ListItem
# list_items = [e for e in elementos if e.category == "ListItem"]

# print(f"Total de ListItem encontrados: {len(list_items)}")

# # Pega os 5 primeiros
# for i, e in enumerate(list_items[:5], 1):
#     print(f"\n--- ListItem {i} ---")
#     print(str(e))



### Vetorizando a base de dados 

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Se você carregou com PyPDFLoader, já tem docs:
# docs = [p.page_content for p in pages]

print(f"Total de blocos extraídos: {len(docs)}")
print(docs[0][:300])  # exemplo do primeiro

# Criando chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,   # tamanho máximo de cada pedaço
    chunk_overlap=20  # sobreposição para manter contexto
)

chunks = text_splitter.split_text("\n".join(docs))

print(f"Total de chunks: {len(chunks)}")
print(chunks[0][:300])


Total de blocos extraídos: 62
Demonstrações Financeiras Completas 
Banco BTG Pactual S.A. e Controladas 
Dezembro 2024
Total de chunks: 518
Demonstrações Financeiras Completas 
Banco BTG Pactual S.A. e Controladas 
Dezembro 2024
www.pwc.com.br 
Banco BTG 
Pactual S.A. 
Demonstrações financeiras
individuais e consolidadas em 
31 de dezembro de 2024
e relatório do auditor independente
PricewaterhouseCoopers Auditores Independentes Ltda., 


### Criando embeddings + Vector Store

In [7]:
import pinecone
print(pinecone.__file__)

c:\Users\Matheus Almeida\OneDrive\Documentos\Insper\Estágio\market_sizing\venv\Lib\site-packages\pinecone\__init__.py


In [8]:
# Pinecone (SDK novo usa import do módulo inteiro)
# LangChain + Pinecone + OpenAI
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

# Inicializa o cliente
load_dotenv()

openai_key = os.getenv("OPENAI_API_KEY")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))


# Cria o índice (só precisa fazer 1 vez, se não existir ainda)
index_name = "meu-indice"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,  # precisa bater com a dimensão do embedding
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# Conecta ao índice
index = pc.Index(index_name)

# Carrega os chunks para o Pinecone
vectorstore = PineconeVectorStore.from_texts(
    texts=chunks,
    embedding=embeddings,
    index_name=index_name
)


print("Base vetorial criada no Pinecone com sucesso!")


c:\Users\Matheus Almeida\OneDrive\Documentos\Insper\Estágio\market_sizing\venv\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


Base vetorial criada no Pinecone com sucesso!


In [ ]:
# Criar retriever a partir do vectorstore
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

# Query
query = "Retorne a linha que aparece Provisão para perdas associadas ao risco de crédito"
docs = retriever.get_relevant_documents(query)

# Mostrar os resultados encontrados
for d in docs:
    print("\n--- Trecho encontrado ---")
    print(d.page_content)

# Pegar o primeiro resultado para expandir contexto
import difflib

match = docs[0].page_content
idx = difflib.get_close_matches(match, chunks, n=1)

if idx:
    i = chunks.index(idx[0])
    contexto = chunks[max(0, i-2): i+3]  # 2 antes e 2 depois
    print("\n=== Contexto expandido ===")
    display(Markdown("\n".join(contexto)))
else:
    print("Nenhum chunk semelhante encontrado.")

# TO do

# Retornar mais preciso -> chunk + contexto 
# Colocar índice em cada documento 
# Rodar localmente para os 5 e ver se o resultado é o esperado (Checar se o valor é o correto e da empresa correta)
# Fazer o teste com mais arquivos 20/30/50 -> Agent Chat GPT OBS: 3 a 5 pdfs sem pdd ou empresas q nn façam sentido 
# Checar saldo x dre; checar números e unidades; chacar se as companhias estão certas;
# Cronometrar tempo 


C:\Users\Matheus Almeida\AppData\Local\Temp\ipykernel_12528\1440387344.py:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)



--- Trecho encontrado ---
Provisão para perdas esperadas associadas ao risco de crédito 10 (1.393.068) (2.240.445) (5.139.334) (5.026.558) 
Outros créditos 89.455.797 33.137.180 119.200.605 55.507.058 
Créditos por avais e fianças honrados 455.622 182.271 458.208 182.614 
Carteira de câmbio 11 62.886.836 15.733.743 59.611.137 16.051.810 
Rendas a receber 12 2.140.228 1.926.797 3.823.691 3.090.093 
Diversos 12 20.274.348 11.945.987 46.712.222 28.341.600

=== Contexto expandido ===


Agrade
cemos aos clientes e parceiros pelo suporte e pela confiança e, em especial aos nossos funcionários, por 
todo o empenho na busca pela excelência.
11 
Demonstrações Financeiras Completas 
Banco BTG Pactual S.A. 
Balanço patrimonial 
(Em milhares de reais)  
Banco Consolidado 
Ativo Nota 31/12/2024 31/12/2023 31/12/2024 31/12/2023      
Disponibilidades 6 1.166.017 1.323.966 4.614.304 2.439.095 
Instrumentos financeiros 400.586.435 358.682.001 505.489.368 420.671.514
Aplicações interfinanceiras de liquidez 7 116.842.450 76.709.394 99.782.456 73.564.489 
Títulos e valores mobiliários 8 154.297.207 167.438.407 188.893.206 180.909.251 
Instrumentos financeiros derivativos 9 43.074.698 40.746.460 26.111.074 19.983.627 
Relações interfinanceiras 16.155.243 15.881.267 33.336.119 23.712.400 
Operações de crédito 10 71.609.905 60.146.918 162.505.847 127.528.305
Provisão para perdas esperadas associadas ao risco de crédito 10 (1.393.068) (2.240.445) (5.139.334) (5.026.558) 
Outros créditos 89.455.797 33.137.180 119.200.605 55.507.058 
Créditos por avais e fianças honrados 455.622 182.271 458.208 182.614 
Carteira de câmbio 11 62.886.836 15.733.743 59.611.137 16.051.810 
Rendas a receber 12 2.140.228 1.926.797 3.823.691 3.090.093 
Diversos 12 20.274.348 11.945.987 46.712.222 28.341.600
Ativos fiscais diferidos 18 4.196.257 3.694.137 9.394.577 8.289.494 
Provisão para redução ao valor recuperável de ativos (497.494) (345.755) (799.230) (448.553) 
Outros valores e bens 3.112.307 1.970.086 3.584.036 2.512.437 
Despesas antecipadas 1.650.862 1.127.497 1.949.531 1.419.358 
Demais 1.461.445 842.589 1.634.505 1.093.079 
Permanente 65.260.243 47.134.904 13.953.493 12.102.745 
Investimentos 64.693.604 46.471.872 9.148.255 7.346.744
Participação em controladas, coligadas e empresas com controle compartilhado 13 64.692.374 46.470.642 9.132.044 7.278.897 
Outros investimentos 1.230 1.230 16.211 67.847 
Imobilizado de uso 14 198.445 188.275 647.908 515.092 
Imóveis de uso 3.349 3.038 13.143 23.608 
Outras imobilizações de uso 525.590 470.835 1.350.960 1.096.927 
Depreciações acumuladas (330.494) (285.598) (716.195) (605.443) 
Intangível 14 368.194 474.757 4.157.330 4.240.909